In [252]:
# -*- coding: utf-8 -*-
import numpy as np
from dewloosh.core.tools import issequence


class CoordinateArrayBase(np.ndarray):
    
    def __new__(subtype, shape=None, dtype=float, buffer=None, 
                offset=0, strides=None, order=None, frame=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super().__new__(subtype, shape, dtype,
                              buffer, offset, strides, order)
        # set the new 'info' attribute to the value passed
        obj.frame = frame
        obj.inds = None
        obj._indsbuf = None
        # Finally, we must return the newly created object:
        return obj
    
    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        #
        # Store indices if obj is a result of a slicing operation
        # and clean up the reference
        self.frame = getattr(obj, 'frame', None)
        if isinstance(obj, CoordinateArrayBase):
            self.inds = obj._indsbuf
            obj._indsbuf = None
        # We do not need to return anything
    
    def __getitem__(self, key):
        key = (key,) if not isinstance(key, tuple) else key
        if isinstance(key[0], slice):
            slc = key[0]
            start, stop, step = slc.start, slc.stop, slc.step
            start = 0 if start == None else start
            step = 1 if step == None else step
            stop = len(self) if stop == None else stop
            self._indsbuf = list(range(start, stop, step))
        elif issequence(key[0]):
            self._indsbuf = key[0]
        elif isinstance(key[0], int):
            self._indsbuf = [key[0]]
        return super().__getitem__(key)
    
    @property    
    def x(self):
        return self[:, 0] if len(self.shape) > 1 else self[0]
    
    @property    
    def y(self):
        return self[:, 1] if len(self.shape) > 1 else self[1]
    
    @property    
    def z(self):
        return self[:, 2] if len(self.shape) > 1 else self[2]

In [253]:
c = np.array([[0, 0, 0], [0, 0, 1.]])

In [254]:
coords = c.view(CoordinateArrayBase)
coords

CoordinateArrayBase([[0., 0., 0.],
                     [0., 0., 1.]])

In [255]:
coords.base is c

True

In [256]:
coords[0,:].base.frame

In [257]:
coords[1, :].z

1.0

In [258]:
coords[0, 0];

In [259]:
coords[:, 0];

In [260]:
coords[0, :];

In [261]:
coords[:, :];

In [262]:
coords[[0, 1], :2];

In [263]:
coords[:2, :].inds

[0, 1]

In [264]:
from numpy.lib.mixins import NDArrayOperatorsMixin

class CoordinateArray(NDArrayOperatorsMixin):
    
    def __init__(self, *args, frame=None, **kwargs):
        buf = np.array(*args, **kwargs)
        self._array = CoordinateArrayBase(shape=buf.shape, buffer=buf, 
                                          dtype=buf.dtype, frame=frame)
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self._array}, frame={self._array.frame})"
    
    def __array__(self, dtype=None):
        return self._array
    
    def __getitem__(self, key):
        return self._array.__getitem__(key)
    
    @property    
    def x(self):
        return self._array.x
    
    @property    
    def y(self):
        return self._array.y
    
    @property    
    def z(self):
        return self._array.z

In [265]:
COORD = CoordinateArray([[0, 0, 0], [0, 0, 1.]])

In [266]:
COORD[:,:].inds

[0, 1]